### Notebook for testing and constructing the retrieval of documents from ifixit
##### Features

- _write_searchphrase_ (arg: search_info, returns: search_phrase): rewriting info (str) into search query
- _load_guides_ (arg: search_phrase, returns: guides): loading guides from ifixit using search phrase, stripping words if no guides are found
- _load_ifixit_guides_ (arg: search_info, returns: guides): both functions combined

In [8]:
#development
from dotenv import load_dotenv
import os
from groq import Groq
import warnings
warnings.filterwarnings("ignore")
#production
from langchain_community.document_loaders import IFixitLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain import hub
from langchain.prompts.prompt import PromptTemplate


In [6]:
#initialize test model
def init_test_model():
    load_dotenv()
    groq_key = os.getenv('GROQ_API_KEY')
    client = Groq(api_key=groq_key)
    return client


In [4]:
#function for rewriting info into searchphrase
def write_searchphrase(search_info: str, debug: bool = False):
    """
    Uses the LLM to rewrite input into a structured searchphrase iFixit searches.

    Args:
        search_info (str): The information to be turned into a searchphrase.
    """
    chat_completion = client.chat.completions.create(
        messages=[
            {"role": "system",
             "content": """Rewrite the following info into a structured searchphrase for iFixit, 
             ensuring it includes the device type, model, broken part. Do not include any other words or comments."""},
            {"role": "user", "content": search_info},
        ],
        model="llama3-8b-8192",
        temperature=0
    )
    search_phrase = chat_completion.choices[0].message.content
    if debug == True:
        print('Full searchphrase:', search_phrase)
    return search_phrase


#load guides from iFixit
def load_guides(search_phrase: str, debug: bool = False):
    """
    Load a guide from IFixit based on the search phrase.
    If no guide is found, iteratively remove the last word and retry.
    
    Args:
        search_phrase (str): The phrase to search for in IFixit guides.
        
    Returns:
        guides: The loaded guide data or None if no guide is found.
    """
    words = search_phrase.split()
    
    while words:
        query = " ".join(words)
        guides = IFixitLoader.load_suggestions(query, doc_type='guide')
        
        if guides:
            if(debug == True):
                print('Used words:', words)
            return guides  # Return results if found
        
        words.pop()  # Remove the last word and retry

    print('No guides found')
    return None  # Return None if no guide is found


#loading guides from iFixit using optimized searchphrase
def load_ifixit_guides(search_info: str, debug: bool = False):


    search_phrase = write_searchphrase(search_info, debug=debug)
    guides = load_guides(search_phrase, debug=debug)
    return guides



In [7]:
#demonstration of the function
client = init_test_model()
guides = load_ifixit_guides("my iphone 6 plus is broken and i need to replace the screen", debug=True) 
print(guides)

Full searchphrase: iPhone 6 Plus Screen Replacement
Used words: ['iPhone', '6', 'Plus', 'Screen', 'Replacement']
[Document(metadata={'source': 'https://www.ifixit.com/Guide/iPhone+6+Plus+Screen+Replacement/30265', 'title': 'iPhone 6 Plus Screen Replacement'}, page_content="# iPhone 6 Plus Screen Replacement\nLearn how to replace your iPhone 6 Plus screen. This part comes with the front-facing camera, earpiece speaker and sensor cable already installed, making for an easier repair.\n\nAll you need to do is remove the old screen and transfer the home button to the new screen, in order for Touch ID (fingerprint scanning) to function.\n\n[video|https://www.youtube.com/watch?v=5w-08cn6ufw]\n\n\n###Tools Required:\n\n - P2 Pentalobe Screwdriver iPhone\n\n - Clampy - Anti-Clamp\n\n - Scissors\n\n - iFixit Opening Tool\n\n - Suction Handle\n\n - Tweezers\n\n - Phillips #000 Screwdriver\n\n\n###Parts Required:\n\n - iPhone 6 Plus Screen\n\n - NuGlas Tempered Glass Screen Protector for iPhone 6 